# Attributes and Functions

The following shows all of the needed attributes and functions to use the package. These are in the that one would tend to need them to perform a calculation. All values shown here are the default values used.

In [ ]:
import bem_electrostatics ## name to use for import

In [ ]:
## Creation of solute object which creates and imports surface mesh, and imports charge values and positions
protein = bem_electrostatics.solute(
    solute_file_path, ## This is the location of pqr file for the solute
    external_mesh_file = None ## Used to define location of external pre-create mesh to be used. If defined without file extension (../../surf_mesh) it is assumed to be an msms mesh (.vert/.face), else it is imported based on the file extension and can be of any type available in meshio
    save_mesh_build_files = False, ## Whether you wish to save the files created during the building and import of the mesh (otherwise this takes place in a tempory folder)
    mesh_build_files_dir = "mesh_files/", ## The directory where to save biuld files, if this is requested
    mesh_density = 1.0, ## If msms is used to mesh this is the mesh density, however, if nanoshaper is used this is the grid_scale (Approximate conversions: density of 4 ≈ grid_scale of 1.6, density of 11 ≈ grid_scale of 2.7, density of 23 ≈ grid_scale of 3.9)
    mesh_probe_radius = 1.4, ## Size of the probe to be used during SES generation
    mesh_generator = "nanoshaper", ## Which porgram to use for gerating the surface mesh. NanoShpaer is inclued with the library, 
    print_times = False, ## Whether to print the time taken for varouis parts of the calculation (time to genertate RHS, time to pass to discrete, etc.)
    force_field = "amber" ## Which force field to use in the generation of a pqr from a pdb (pdb2pqr must be installed for this)
)

In [ ]:
## The following attributes of the created object are used to save the arguments specified during the object's creation
protein.save_mesh_build_files
protein.mesh_build_files_dir
protein.mesh_density
protein.mesh_probe_radius
protein.mesh_generator
protein.force_field

## As either a pdb or pqr can be specified, the following attribute shows which was used
protein.imported_file_type
## If a pdb is used the path is saved to:
protein.pdb_path
## else the used pqr path is saved to:
protein.pqr_path

## The solute name is saved to the following attribute. If a pdb is used it is taken from this, otherwise it corresponds to the pqr filename
protein.solute_name

In [ ]:
protein.mesh ## Attribute contining the bemmp mesh object of the solute
protein.q ## Attribute containing the pioint charges present in the the solute
protein.x_q ## Attribute 

protein.mesh.number_of_elements

In [ ]:
## There are three different formulations avaible to solve the potential across the boundary
## These are "direct" (linearized Poisson–Boltzmann equation), "juffer" (see Juffer ETal 1991) and "alpha_beta" (currently under analysis)
## The formulation to be used is set with the following attribute:
protein.pb_formulation = "direct"

## Attributes for the internal and external dielectric constants and inverse Debye length (kappa)
protein.ep_in = 4.0
protein.ep_ex = 80.0
protein.kappa = 0.125

## Attributes for the aplha and beta values to be used in the case that this formulation is used
protein.pb_formulation_alpha = 1.0
protein.pb_formulation_beta = self.ep_ex/self.ep_in

## Whether to apply or not calderon preconditioning (only if using alpha_beta), and which to apply (squared, interior or exterior)
protein.pb_formulation_preconditioning = False
protein.pb_formulation_preconditioning_type = "squared"

## Select which discrete form to use (strong or weak)
protein.discrete_form_type = "strong"

## Attributes that can be changed for the GMRES solver
protein.gmres_tolerance = 1e-5
protein.gmres_max_iterations = 1000

In [ ]:
protein.calculate_potential() ## Function to solve the potential across the boundary using the parameters set above (formulation, dielectric constants, etc.)

In [ ]:
## Attributes to whcih the results are saved
protein.solver_iteration_count ## Iteration count of the GMRES solver
protein.phi ## Bempp grid function of the potential on the surface mesh
protein.d_phi ## Bempp grid function of the derivative of the potential on the surface mesh

In [ ]:
protein.calculate_solvation_energy() ## Function to calculate the solvatation energy of the solute, using the results of the surface potential calculation

In [ ]:
## Attributes to whcih the results are saved
protein.solvation_energy ## Solvatation energy calculated, in [kcal/mol]

---

# Example 1

Now a simple example of how to calulate the solvatation energy of a protein using an previously generated pqr file. In this case it will be the 5pti protein having generated the pqr from the pdb file using the CHARMM force field. First we must import the main package.

In [1]:
import bem_electrostatics

Could not find Gmsh.Interactive plotting and shapes module not available.


Now we create solute object, giving the path of the pqr file to be used.

In [2]:
protein = bem_electrostatics.solute("5pti.pqr")

If a previously created msms or other type of mesh is to be used, we can import it as follows.

In [ ]:
#protein = bem_electrostatics.solute("5pti.pqr" , external_mesh_file = "surf_d02") ## If no extension is found it is assumed to be .vert/.face

#protein = bem_electrostatics.solute("5pti.pqr" , external_mesh_file = "surf_d02.off") ## Or any file supported by meshio can be used

Now we have gerated and imported the mesh, as well as the charges and their positons. As the mesh is a bempp mesh object we can manipulate it with the corresponding functions for this object.

In [3]:
print("Number of elements:", protein.mesh.number_of_elements)
print("Number of vertices:", protein.mesh.number_of_vertices)
#protein.mesh.plot() ## Uncomment to see a plot of the surface mesh

Number of elements: 9352
Number of vertices: 4676


As we will calculate the energy using all of the default options we only need call the calculate_solvation_energy() function.

In [4]:
protein.calculate_solvation_energy()

/home/stefan/miniconda3/envs/bempp/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:296: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


This has first calculated the potential on the surface mesh, then calculated the solvation energy of the protein which we can print as follows:

In [5]:
print("Solvation energy:", protein.solvation_energy, "[kcal/mol]")

Solvation energy: -338.37565198177055 [kcal/mol]


---

# Example 2

Now an example of how to calulate and visulalise only the potential across the surface.

In [6]:
import bem_electrostatics

Create the solute object using the same pqr as before.

In [7]:
protein2 = bem_electrostatics.solute("5pti.pqr")

Now to calculate the surface potential using the default parameters, all we have to do is call the calculate_potential() function.

In [ ]:
protein2.calculate_potential()

/home/stefan/miniconda3/envs/bempp/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:296: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


To visulise the surface potiential we apply the plot() funtion to the calculated surface potential (phi), which is saved as a bempp GridFunction.

In [ ]:
protein2.phi.plot()